## Baseline

In [ ]:
from pipeline.baseline import run_baseline, process_csv_to_numpy

def main():
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_csv_path = "result/discrete_scores.csv"
    model_name = "gpt-3.5-turbo"

    run_baseline(file_path_excel, file_paths_csv, result_csv_path, model_name)

if __name__ == "__main__":
    # main()

    # Path to the input CSV file
    input_csv_path = "result/discrete_scores.csv"  # Replace with your actual CSV file path

    # Path to save the resulting numpy file
    output_numpy_path = "result/p0_matching_matrix.npy"  # Replace with your desired output path

    # Process the CSV and save to numpy file
    process_csv_to_numpy(input_csv_path, output_numpy_path)

## Pipeline 1

In [ ]:
from pipeline.pipeline_1 import run_pipeline_1

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_path = "result/p1_matching_matrix.npy"
    embedding_cache_path = "embedding/Pipeline_1_embeddings_cache.pkl"

    # Run Pipeline 1
    run_pipeline_1(file_path_excel, file_paths_csv, embedding_cache_path, result_path)


if __name__ == "__main__":
    main()


## Pipeline 2

In [ ]:
from pipeline.pipeline_2 import run_pipeline_2

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    embedding_cache_path = "embedding/Pipeline_2_embeddings_cache.pkl"
    result_path = "result/p2_matching_matrix.npy"

    # Set hyperparameters
    top_k = 20  # Number of top reranked matches to retrieve
    bi_top_k = 40  # Number of top Bi-Encoder matches before reranking

    # Run Pipeline 2
    run_pipeline_2(file_path_excel, file_paths_csv, embedding_cache_path, result_path, top_k=top_k, bi_top_k=bi_top_k)


if __name__ == "__main__":
    main()

## Pipeline 3

In [ ]:
from pipeline.pipeline_3 import run_pipeline_3
import nest_asyncio

# 允许嵌套事件循环
nest_asyncio.apply()

def main():
    # Define file paths
    file_path_excel = "2024-09-27_Food_Waren-und_Dienstleistungsgruppe_V_4.0.xlsx"
    file_paths_csv = "grouped_result.csv"
    result_path = "result/p3_matching_matrix.npy"
    embedding_cache_path = "embedding/Pipeline_3_embeddings_cache.pkl"

    # Set hyperparameters
    model_name = "gpt-4o-mini"  # Specify GPT model

    # Run Baseline Pipeline
    run_pipeline_3(file_path_excel, file_paths_csv, embedding_cache_path, result_path, gpt_model_name=model_name)


if __name__ == "__main__":
    main()

# Evaluation

In [ ]:
from pipeline.evaluate import recall_k, unrelated_acc, f_distribution_detection
import numpy as np
import statistics

# 目标矩阵路径
target_matrix_path = 'matched_table.csv'
full_target_matrix = np.loadtxt(target_matrix_path, delimiter=",", dtype=int)
target_matrix_60 = full_target_matrix[:60]  # 截取前 60 行

for i in range(6):
    # 这三个列表分别用于存储 p{i} 的三次重复实验所得的指标
    unrelated_scores = []
    recall_scores = []
    f_accuracies = []

    for rep in range(1, 4):
        # 读取 p{i}_matching_matrix_{rep}.npy
        predicted_matrix_path = f"result/p{i}_matching_matrix_{rep}.npy"
        predicted_matrix = np.load(predicted_matrix_path)

        # 同样截取前 60 行
        predicted_matrix_60 = predicted_matrix[:60]

        # 计算指标
        unrelated_score = unrelated_acc(predicted_matrix_60)
        recall_score = recall_k(predicted_matrix_60, target_matrix_60)
        f_accuracy = f_distribution_detection(predicted_matrix_60, target_matrix_60, alpha=0.05)

        unrelated_scores.append(unrelated_score)
        recall_scores.append(recall_score)
        f_accuracies.append(f_accuracy)
    
    # 计算三次重复实验的均值和标准差
    mean_unrelated = statistics.mean(unrelated_scores)
    std_unrelated = statistics.pstdev(unrelated_scores)
    
    mean_recall = statistics.mean(recall_scores)
    std_recall = statistics.pstdev(recall_scores)
    
    mean_f = statistics.mean(f_accuracies)
    std_f = statistics.pstdev(f_accuracies)

    # 打印结果，格式如 2066.009±15.907
    print(f"===== Pipeline p{i} =====")
    # print(f"Unrelated Accuracy: {mean_unrelated:.3f}±{std_unrelated:.3f}")
    print(f"Recall@K:          {mean_recall:.3f}±{std_recall:.3f}")
    print(f"F Accuracy:        {mean_f:.3f}±{std_f:.3f}\n")


===== Pipeline p0 =====
Unrelated Accuracy: 0.500±0.000
Recall@K:          0.046±0.023
F Accuracy:        0.511±0.021

===== Pipeline p1 =====
Unrelated Accuracy: 0.500±0.000
Recall@K:          0.527±0.000
F Accuracy:        0.550±0.000

===== Pipeline p2 =====
Unrelated Accuracy: 0.000±0.000
Recall@K:          0.083±0.000
F Accuracy:        0.383±0.000

===== Pipeline p3 =====
Unrelated Accuracy: 0.722±0.028
Recall@K:          0.340±0.069
F Accuracy:        0.572±0.016

===== Pipeline p4 =====
Unrelated Accuracy: 0.589±0.061
Recall@K:          0.202±0.153
F Accuracy:        0.567±0.036

===== Pipeline p5 =====
Unrelated Accuracy: 0.617±0.083
Recall@K:          0.098±0.060
F Accuracy:        0.539±0.028



/home/fcr/.local/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:573: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)
